[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/atmos-cloud-sim-uj/PySDM.git/master?filepath=PySDM_tutorials/environments/kinematic_2d.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atmos-cloud-sim-uj/PySDM/blob/master/PySDM_tutorials/environments/kinematic_2d.ipynb)

In [8]:
"""
clone and install PySDM dependencies in Colab
(to use GPU on Colab set hardware accelerator to 'GPU' before session start
in the "Runtime :: Change runtime type :: Hardware accelerator" menu)
"""
import os, sys
if 'google.colab' in sys.modules:
    %cd /content
    if not os.path.isdir('PySDM'):
        !git clone --depth 1 https://github.com/atmos-cloud-sim-uj/PySDM.git
    %cd PySDM
    !pip install -r requirements.txt

In [9]:
import os, sys
if 'google.colab' in sys.modules:
    %cd /content/PySDM
else:
    sys.path.insert(0, os.path.join(os.getcwd(), '../..'))

In [10]:
import numpy as np

from PySDM.physics import si
from PySDM.backends import CPU
from PySDM.dynamics import Displacement
from PySDM.dynamics import EulerianAdvection
from PySDM.dynamics import AmbientThermodynamics
from PySDM.environments.kinematic_2d.arakawa_c import make_rhod, nondivergent_vector_field_2d
from PySDM.environments.kinematic_2d.mpdata import MPDATA
from PySDM.environments import MoistEulerian2DKinematic
from PySDM.products.environments import RelativeHumidity
from PySDM.initialisation import spectral_sampling, spatial_sampling
from PySDM.initialisation.spectra import Lognormal
from PySDM.builder import Builder
from PySDM.products.state import SuperDropletCount
from PySDM.physics.formulae import th_dry


In [11]:
rhod_of = lambda zZ: 1+zZ*0  # TODO
field_values = {'qv': .5 * si.g / si.kg}
field_values['th'] = th_dry(289 * si.K, field_values['qv'])


In [12]:
builder = Builder(n_sd=1, backend=CPU)
environment = MoistEulerian2DKinematic(
    dt=1*si.s,
    grid=(64, 64),
    size=(1 * si.km, 1 * si.km),
    rhod_of=rhod_of
)

In [13]:
def stream_function(x, z):
    return  (.6 * si.m / si.s * (si.kg / si.m ** 3)
             * environment.mesh.size[0]
             / np.pi * np.sin(np.pi * z) * np.cos(2 * np.pi * x))

In [14]:
g_factor = make_rhod(environment.mesh.grid, environment.rhod_of)
advector = nondivergent_vector_field_2d(environment.mesh.grid, environment.mesh.size, environment.dt, stream_function)
advectees = dict((key, np.full(environment.mesh.grid, value)) for key, value in field_values.items())



mpdatas = MPDATA(advectees=advectees, g_factor=g_factor, advector=advector)
environment.set_advection_solver(mpdatas)
builder.set_environment(environment)

builder.add_dynamic(AmbientThermodynamics())
builder.add_dynamic(EulerianAdvection())
builder.add_dynamic(Displacement())

attributes = environment.init_attributes(
    spatial_discretisation=spatial_sampling.Pseudorandom(),
    spectral_discretisation=spectral_sampling.ConstantMultiplicity(
        spectrum=Lognormal(norm_factor=60 / si.mg, m_mode=0.04 * si.um, s_geom=1.4),
    ),
    kappa=1 * si.dimensionless)
products = [SuperDropletCount(), RelativeHumidity()]
core = builder.build(attributes, products)

for _ in range(10):
    core.run(steps=10)